# Finding Function in Form: Compositional Character Models for Open Vocabulary Word Representation

Ling2015

## 0. あぶすと
双方向LSTMを用いて文字を構成化することで単語のベクトル表現を行うモデルを提案する  
各単語の形ごとに独立したベクトルを持つこれまでの単語表現モデルと比較して、提案モデルは文字形ごとの単一ベクトルと構成モデルのためのパラメータの固定セットのみを必要とする  
提案モデルの単純さ、そしてさらに言語における語形と機能の関係の恣意的性質にもかかわらず、この"構成的な"単語表現は、言語モデルおよび品詞のタグ付けにおいてもっとも良い結果を残した  
従来のベースラインと比較して、形態的に高級な言語において特に有用性が顕著である  

## 1. いんとろ
自然言語処理の応用の一般化には、単語の表現が重要となる  
その重要さの根幹をなすのは、幾何的局所性が(意味的・構文的な)類似性をよくとらえたようなベクトル空間モデルの構築である  
しかし、単語ベクトルを学習する際、ほとんどのモデルでは各単語は他の構成要素と独立して変化するような独自のベクトル表現を有していると仮定している  
本紙では、この独立性の前提が、形態的に高級な言語(トルコ語のような)において問題であると主張する  
このような言語では、より正確な仮定として、文法的に正しく類似した表現は機能的に類似している証であるということが挙げられる  
しかし、字面の類似が機能の類似にとって必要十分条件でないことは明らかである  
小さな正書法の違いが大きな意味・構文の違いとなることもあれば(batterとbutter)、大きな正書法の違いが近い意味に成ることもある(richとaffluent)  
したがって、正書法に重きをおいたモデルは、形態素のような規則的な影響に加えて、非構成的な影響も考慮する必要がある  
このような複雑な単語形と機能の関係をモデル化するため、ある系列における複雑な非線形・非局所ダイナミクスを連続して取り込めるよう設計されたLSTMを用いる  

提案モデルは、双方向LSTMを用いて各単語の文字列を"読み"、そしてそれらを組み合わせることにより単語のベクトル表現とする  
このモデルでは、各文字形がベクトルに関連付けられていることを仮定し、LSTMのパラメータは特定語彙や通常の形態学の知識の両方を符号化する  
モデルの評価を行うため、品詞タグ付けと言語モデリングのためのベクトルベースドモデルを用い、正字法を用いないパラメータ設定を使用するこれまでのベースラインと比較し、様々な言語での実験を報告する  
実験では以下の4点が示された  
 1. 提案モデルが意味的・構文的に同じ意味である単語について同じような表現を生成できる
 1. 単語同士が正書法上離れていても、提案モデルは二つのタスクにおいてワードルックアップテーブルよりも少ないパラメータを用いて、より改善された  
 1. 提案モデルは品詞タグ付けにおいて(英語での新記録を含む)最高位の性能を保持している  
 1. 形態学的に高級な言語における劇的な性能の改善  



## 2. 単語ベクトルと単語なし単語ベクトル
語彙Vにおけるすべての単語が他の単語と同じ距離であるような単純なモデルと対照的に、ベクトル空間モデルは単語は様々な次元に沿って異なっていたり、類似していたりするという直感をとらえている  
様々な種類の言語モデルにおいて、単語のベクトル表現を最適化可能なパラメータとして扱い学習することは、その他の作業を円滑に進めるベクトル表現を生成するための有効な手段として知られている  
このようなモデルは、一般に語彙Vの各単語についてdのパラメータを含む行列$\mathbf{P}\in \mathbb{R}^{d\times|V|}$で定義される  
$w\in V$があたえられたとき、長さ$|V|$のone-hotベクトル($\mathbf{1}_w$: wに関連する要素を除くすべての次元が0)を$\mathbb{P}$の右から掛け合わせることで列を選択する   
行列$\mathbf{P}$は単語ルックアップテーブルとも呼ばれる(Pから得られた埋め込みベクトルe)  
単語wについて単語ルックアップテーブルから得られた埋め込みは$e^W_w\in \mathbb{R}^d$で表す($e^W_w= \mathbf{P}\bullet \mathbf{1}_w$)  
これにより、アノテートされたデータが少ないタスクであっても、大量のデータを持つ他のタスクと共同して訓練し、単語の類似性を活用することが可能と成る  
この手法を実現する基本的な方法として、教師なしタスクにより訓練されたパラメータを用いて単語ルックアップテーブルを初期化する方法がある  
例: skip-n-gram, CBoW

### 2.1 問題点: 独立パラメータ
単語ルックアップテーブルには二つの問題点がある  
一つは、単語間の意味的・構文的類似性を学ぶには大量のデータによる事前訓練を必要とするが、各ベクトルは独立している  
つまり、単語ルックアップテーブルに基づくモデルは"cat","king","queen"と"cats","kings","queens"がほぼ同じ線形対応関係にあることは学習するが、このモデルでは単語の最後にsを追加することが変形の証であるという事実が表現されていない  
これは、単語ルックアップテーブルは、別の文脈で用いられる既存単語を組み合わせたような未知語の表現を行うことができない  
ふたつ目は、たとえコーパスデータが有効であっても、すべての単語をベクトルに保持することは現実的でない  
比較的形態素が貧弱である英語であっても、語の数は数十万まで増加する傾向があるため、オンラインデータのようなノイズの多い領域では語形は大幅に増加する  
たとえば、6000万文の英語版ウィキペディアダンプでは、それぞれに独立なベクトルを必要とする約2000万の異なる小文字トークン化された語形がある  
そして、認知学者の間では、語彙が関連する項目ごとに構成されている、つまりパラメータが独立ではないと物議を醸していることに留意せねばならない  
コネクショニストとシンボル至上主義者(?)との間でよく知られている"past tense debate"は、人間がどのように語尾変化を表現するか(たとえば、英語の過去時制の形成)という意見についての不一致に関わっている  

### 2.2 解決： 合成モデル
この問題を解決するため、単語のより小さな部分をより広域の表現に構成することにより単語のベクトル表現を構成することを考える  
この考えは、形態素を単語の表現に構成するという先行研究により模索されていた  
形態素は定義上もっとも原子的な意味を持つ(あるいは構文を持つ)とされる言語単位であるため、モデルのための理想的な原子的要素である  
しかし、このようなアプローチでは形態素解析器に依存してしまうことが欠点として考えられる  
そこで、文字の表現を単語の表現に拡張することを提案する  

-ingやら-lyみたいな形態素を語幹にくっつけるといった形態的な行いは比較的規則的に影響を与えるなど、単語の構成における関係は存在するが、その一方で同じような語形を持つ単語でも異なる意味を持つということもある  
ゆえに、単語の形と意味間の関係はさほど重要ではない  


## 3. C2Wモデル
合成文字からの単語モデル(C2W)は双方向LSTMに基づくモデルであり、これは複雑な非局所的依存関係を学習することができるシーケンスモデルである  
C2Wモデルの入力は一つの単語形wであり、このwのd次元べクトル表現を得ることを考える  
このモデルの入力と出力は単語ルックアップテーブルと同一であるため、これらのネットワークの置換は容易に行える  
入力として、文字Cを定義する  
英語の場合、語彙には大文字と小文字からなるエントリと数字と句読点が含まれる  
入力単語であるwは文字列$c_1, ... ,c_m$(mは単語wの長さ)に分解される
$c_i$それぞれをone-hotベクトル$\mathbf{1}_{c_i}$と定義する。語彙Mにおける$c_i$はただひとつ定まる  
次に、射影レイヤ$\mathbf{P}_C\in \mathbb{R}^{d_C\times|C|}$を定義する、ここで$d_C$は文字セットCの次元数を表す  
これは文字ルックアップテーブルであり、ある言語における文字間の類似(母音と子音のような)をつかむために用いられる  
したがって、各入力文字$c_i$についての写像は$\mathbf{e}_{c_i}=\mathbf{P}_C\bullet \mathbf{1}_{c_i}$とかける  
入力ベクトル$\mathbf{x}_1, ... ,\mathbf{x}_m$が入力として与えられると、LSTMは下記の更新を繰り返し適用することで状態系列$\mathbf{h}_1, ... ,\mathbf{h}_{m+1}$を計算する  

$$\mathbf{i}_t = \sigma( \mathbf{W}_{ix}\mathbf{x}_t + \mathbf{W}_{ih}\mathbf{h}_{t-1} + \mathbf{b}_i)$$
$$\mathbf{f}_t = \sigma( \mathbf{W}_{fx}\mathbf{x}_t + \mathbf{W}_{fh}\mathbf{h}_{t-1} + \mathbf{b}_f)$$
$$\mathbf{c}_t = \mathbf{f}_t\odot\mathbf{c}_{t-1} + \mathbf{i}_t\odot tanh(\mathbf{W}_{cx}\mathbf{x}_t + \mathbf{W}_{ch}\mathbf{h}_{t-1}+\mathbf{b}_c)$$
$$\mathbf{o}_t = \sigma(\mathbf{W}_{ox}\mathbf{x}_t + \mathbf{W}_{oh}\mathbf{h}_{t-1} + \mathbf{W}_{oc}\mathbf{c}_t + \mathbf{b}_o)$$
$$\mathbf{h}_t = \mathbf{o}_t\odot tanh(\mathbf{c}_t)$$

ここで$\sigma$は要素ごとの線形シグモイド関数であり、$\odot$は要素ごとの積(アダマール積)である  
LSTMは外部セルメモリ$\mathbf{c}_t$が定義され、各タイムスタンプtにおいて線形に結合される  
$\mathbf{c}_t$から$\mathbf{c}_{t-1}$に伝播された情報は、$\mathbf{i}_t,\mathbf{f}_t,\mathbf{o}_t$という3つのゲート素子により制御され、これらは入力系列$\mathbf{x}_t$のうち何を受け取るか、$\mathbf{c}_{t-1}$から忘却すべきもの、現在状態$\mathbf{h}_t$に関連するものというように定義されている  
LSTMのすべてのパラメータ($\mathbf{W}_{ix},\mathbf{W}_{fx},\mathbf{b}_f, ...$)を参照する記号として、$\mathcal{W}$を記述する  
したがって、入力系列として文字表現$\mathbf{e}^C_{c_1}, ... ,\mathbf{e}^C_{c_m}$が与えられると、順方向LSTMは状態系列$\mathbf{s}^f_0, ... ,\mathbf{s}^f_m$を生成し、一方で逆方向LSTMは入力を逆の系列として受け取り、状態$\mathbf{s}^b_m, ... ,\mathbf{s}^b_0$が生成する  
2つのLSTMは異なるパラメータセットを用いる  
単語wに関する単語表現は以下のような順方向と逆方向の組み合わせで与えられる  
$$\mathbf{e}^C_w = \mathbf{D}^f\mathbf{s}^f_m + \mathbf{D}^b\mathbf{s}^b_0 + \mathbf{b}_d$$


### 効率的なキャッシュ
$e^W_w$と比較して、$e^C_w$は二つの長さmのLSTMを横断するため計算コストは高い  
しかしながら$e^C_w$は単語の文字列のみに依存するため、つまるところパラメータが更新されない限り、繰り返し用いられる異なる単語wごとの値を取得することが可能に成る  
したがって、このモデルは最も頻繁に出現する単語形のリストをメモリに保持し、まれな単語に対してのみ構成モデルを実行できる  
すべての単語をキャッシュすると単語ルックアップテーブルと同じくらいの性能が得られるが、同時に大量のメモリを食いつぶしてしまうことは明白である   
したがって、キャッシュで用いられる単語の数は特定のアプリケーションのメモリ対性能要件を満たすように調節される  
  
訓練時、パラメータが更新される際に、同じバッチ内で繰り返し用いられる単語は一度のみ計算する必要があるが、出力の勾配はバッチ内で累積することが可能であるため、単語形ごとに一回の更新を行う必要がある  
このため、より大きなバッチサイズを定義することがのぞましい  



## 4. 実験: 言語モデル
提案モデルは単語からの文合成表現を計算するために使われるモデルと似ている  
しかし、個々の単語の意味とフレーズや文の複合的な意味間の関係は、文字表現と単語の意味の関係間より間違いなくより規則的である  
ほんまに学習できるんかいな  
よい言語モデルには語順のような構文的側面のみならず意味的側面が要求される  
したがって、もし提案モデルが接頭辞や接尾辞のような単語の規則的な側面のみを取り込むなら、単語ルックアップテーブルと比較してより悪い結果になるだろう  

### 4.1 言語モデル
言語のモデル化は対数確率$\log p(\mathbf{w}) , \mathbf{w}=(w_1, ... ,w_n) $を計算する関数を学習することにつながる  
この計算値はチェーンルールに従って条件付き対数確率の積和に分解することができる  

$$\sum^n_{i=1}\log p(w_i | w_1, ... ,w_{i-1})$$

提案する言語モデルでは、この$\log p(w_i | w_1, ... ,w_{i-1})$をリカレントLSTMを用いた単語の合成表現$w_1, ... ,w_{i-1}$から計算する  
モデルを図に表した図２において、各単語$w_i$が単語表現に写像される第一段階に着目する  
これは単語ルックアップテーブル$\mathbf{e}^{\mathbf{W}}_{wi}$を用いることで可能となり、この場合通常のリカレント言語モデルが作成される  
提案モデルを使う場合、単語ルックアップテーブルを提案モデル関数$f(w_i) = \mathbf{e}^C_{w_i}$に容易に置き換えられる  
各LSTMブロック$s_i$は、単語$w_{i+1}$を予測するために用いられる  
これは$s_i$を語彙Vのサイズのベクトルに射影し、ソフトマックス関数を適用することによって実行される  
ソフトマックス関数は単純な$d\times V$のテーブルであり、与えられた文脈における単語の出現確率を符号化する、クローズド語彙モデルである  
したがって、テスト時には語彙外(out-of-vocabulary)単語は対処することができない  
一般に用いられる戦略として、低頻度の語句をOOVトークンとして置換することによる語彙Vの単語刈りがある  
テスト時、未知語の確率はOOVトークンとして推定される  
したがって、刈り取られた単語の数に応じて、ソフトマックス関数から得られる結果が少なくなるため、全体のperplexity値が減少するおそれがある  
よって、異なる語彙サイズのモデルを比較する際、perplexityの絶対値は有益ではない  
(perplexity: 確率の逆数、選択肢の数　モデルM下で正解を選ぶ難しさを表す→モデルの予測性能を表す)  
しかし、異なるモデル間の相対的なperplexityはどのモデルが文脈に基づいてより正確に単語を予測できるかということを示している  
ベースラインの設定ではOOVワードに対処するため、これらは未知トークンに置き換えられ、埋め込みセットに関連付けられる  
訓練中、一度しか表れない単語は0.5の確率で確率的に置き換えられる  
同様の処理が、C2Wモデルでも文字レベルで実行される  

### 4.2 実験
#### データセット
英語・ポルトガル語・カタロニア語・ドイツ語・トルコ語といった広域の形態類型論を有する言語について、言語モデルの性能を調べる  
これらの言語はすべて語形変化を含む言語であるが、膠着語(トルコ語)では接辞が変化しない傾向があるのに対し、屈折語(英語)はそうではない(変わる？)  
これらの言語について、Wikipediaの記事から無作為に抽出し、その結果得られた100万単語を訓練に使用した  
開発とテストには更に抽出した2万語を用いた  

#### 実験設定
単語の表現次元dを50に定義する  
C2Wモデルでは文字の次元$d_C$と現在の状態$d_{CS}$の設定が必要となる。実験では$d_C=50,d_{CS}=150$に設定した  
言語モデル列$s_i$で用いられるLSTMの各状態は、状態・セルメモリが共に150に設定される  
訓練は100文ごとのミニバッチ勾配降下法で実行される  
学習レート(勾配の進む距離の大きさ)は0.2、Momentum(慣性: )は0.95に設定された  
ソフトマックス関数は常に小文字単語でのみ実行される  
また、出力する語彙を最も頻繁な5000語に制限する  
残った単語形は未知トークンとして置き換えられ、予測される  
単語表現レイヤはすべての単語形に対し動作する(完全なopen vocabularyでも)  
単語ルックアップテーブルを用いる際、入力単語はすべて小文字化される  
C2Wモデルでは、ケース情報が保持される(?)  
評価はテストデータ上のperplexitiesを計算することにより行われ、開発データにおいて最も高いperplexityを示したパラメータが用いられる  

#### Perplexities
テストセットにおけるPerplexitiesを表1に示す  
これらの結果より、一般的にC2Wモデルは単語ルックアップテーブルよりも常に優れていることが明らかであり、単語の意味の違いが使用される接尾辞に絶対的に依存するような高級な形態言語であるトルコ語では、特によく改善が見られた(evde → in the house / evden → from the house)  

#### パラメータの数
パラメータの数について、単語ルックアップテーブルのパラメータの数は$V\times d$である  
もし言語が80000単語(形態的に高級な言語におけるおおよその推定)を含んでいるとすれば、400万パラメータが必要と成る  
一方で、構成モデルは$d_{CS}\times d_{C}+2d_{CS}$の八次元の行列から成る  
加えて、サイズ$d\times 2d_{CS}$の順方向と逆方向を組み合わせた行列も存在する  
したがって、パラメータの数はおおよそ15万、実際はそれよりも少ない  
このモデルも各エントリごとに$d_C$パラメータの文字ルックアップテーブルを必要とする  
英語では、用いられる文字は618文字であり、30900パラメータが加えられる  
よって、英語の総パラメータ数はおおよそ18万(各単語ごとに2から3パラメータ)であり,単語ルックアップテーブルよりも遥かに少ない  

#### 性能
効率性という観点からすれば、提案モデルと単語ルックアップテーブルはどちらも訓練中に約300語/秒のレートで文にラベル付を行うことができる  
この驚くべき点は、C2Wモデルは文字に渡る構成を必要とするため、システムの主要なボトルネックは語彙におけるソフトマックス関数であることだ  
さらに、同じバッチ内で同じ単語を2回構成するのを避けるため、キャッシュが使用されている  
これが意味するところは、この提案モデルがソフトマックスのような比較的高速なシステムであることを示している  

#### (臨時)単語の表現
提案モデルが単なる語彙特徴を学習するだけではないことは確たる事実であるが、興味深いことに、提案モデルは臨時語を埋め込むことが可能である  
表2に、2つの語彙語と2つの臨時語について、提案モデルによって計算されたコサイン類似度が最も類似した語彙語を示す  
これにより、一般にOOV単語をUNKトークンに置き換えるルックアップテーブルよりも、モデルを一般化することについて大幅に改善することができる  
さらに、この(臨時語を)一般化する能力は人間のそれに酷似しており、人間の形態に基づいて新しい単語の意味を推論することができる  



## 5. 実験： 品詞タグ付け
提案モデルの有用性の第二の例として、品詞タグ付けを目的とする    
形態素は多くの言語で非常に有力な指標であるため、その特徴選択に多くの努力が費やされてきた  
本章では提案モデルを使用してこれらの特徴を自動的に学習できることを示す  

### 5.1 Bi-LSTMタグ付けモデル
提案するタグ付けモデルは斬新であると同様に、非常に単純である  
入力は単語特徴系列$f(w_1), ... ,f(w_n)$であり、これらの単語ベクトルはC2Wモデル$f(w_i) = \mathbf{e}^C_{wi}$もしくは単語ルックアップテーブル$f(w_i) = \mathbf{e}^W_{w_i}$によって生成される  
また、手作業で設計された特徴の使用をテストする、この例を$f_1(w_i), ... ,f_n(w_i)$とする  
次に、順次的に特徴が双方向LSTMに供給され、順方向状態$s^f_0, ... ,s^f_n$と逆方向状態$s^b_{N+1}, ... ,s^b_0$が得られる  
したがって、状態$s^f_i$は0からiまでのすべての単語情報とnからiまでの単語情報$s^b_i$を含む  
順方向と逆方向の状態を組み合わせ、1からn番目のそれぞれについて次の値を得る  
$$\mathbf{l}_i = tanh(\mathbf{L}^f\mathbf{s}^f_i+\mathbf{L}^b\mathbf{s}^b_i+\mathbf{b}_l)$$
ここで、$\mathbf{L}^f,\mathbf{L}^b,\mathbf{b}_l$はそれぞれどのように順方向と逆方向の状態を組み合わせるかといったパラメータである  
順方向$s^f$と逆方向$s^b$状態と組み合わせ状態$\mathbf{l}$の長さはハイパーパラメータ$d^f_{WS},d^b_{WS},d_{WS}$として示される  
そして、i番目の出力ラベルが結合状態$\mathbf{l}_i$の射影により、品詞タグセット上のソフトマックスとして得られる  

### 5.2 実験
#### データセット
英語では、PannによるWall Street Journalデータセットを標準分割(sec1-18を訓練に、sec19-21を開発に、sec22-24をテストに)して、実験を行った  
また、他の4言語についてはCoNLL共有タスクより得られたデータセットを用いて行った  
PTB(Pann Tree Bank)データ・セットからは標準の訓練・チューン・テスト分割を得られるが、他の言語のデータセットではチューニングデータが得られないため、訓練データの最後の100文を抜き出してチューニングに使用する  

#### 設定
品詞モデルは2つのハイパーパラメータのセットを必要とする  
初めに、単語は連続分散表現に変換せねばならず、4章の言語モデルで用いられたものと同じハイパーパラメータを用いる必要がある  
次に、単語表現を結合し文脈を符号化する  
提案する品詞タグ付け器はLSTMの状態に関連する3つのハイパーパラメータ$d^f_{WS},d^b_{WS},d_{WS}$を持ち、そのすべてが50に設定されている  
訓練アルゴリズムについては、言語モデルで用いられたものと同じ設定(学習率、学習勾配、ミニバッチサイズ)を用いる  
単語ルックアップテーブルを用いる設定では、OOV単語をUNKトークンに置きかえ、またC2WモデルではOOV文字をUNKトークンに置き換える  
事前に訓練された単語埋め込みを使用する設定では、ラベル付された訓練データに含まれない単語、および事前訓練時に用いられたラベル付されてない単語はOOVとみなす  

#### 合成モデルにおける比較
C2Wモデルについて、複数の異なるRNNを用いた比較を行った  
すべての実験においてC2Wモデルを用い、PTBデータセットの結果をまとめた  
テストセットにおける精度を"acc"列に示し、単語構成モデルのパラメータ数を"parameter"列に示し、そしてテスト時に処理された1秒あたりの単語の数
を"words/sec"に示す  
この結果を見ると、RNNを用いたアプローチはLSTMのそれよりもおよそ2%ほど精度が悪くなっていることがわかる  
これは、通常のRNNが短い文字系列長の依存を学習することができるが、長い依存関係を学習するには理想的でないことを示唆している  
一方、LSTMは順方向や逆方向を個別に使用する場合であっても、単語ルックアップテーブルを使用する場合と比較して比較的高い結果を効果的に得られている  
双方向LSTMでは最も良い結果が得られ、単語ルックアップテーブルを凌駕している  
PTBデータセットにおける訓練データにはおよそ40Kの単語形が含まれている  
したがって、各単語ごとに50次元の単語ルックアップテーブルではおよそ2百万のパラメータが含まれる  
対してC2Wでは、含まれる文字の種類は(大文字小文字合わせて)およそ80である  
したがって文字ルックアップテーブルには4Kのパラメータのみが含まれ、これは構成モデルにおける150Kパラメータに比べれば無視できる範囲のものである  
この結果において、Bi-LSTMは他モデルよりも多くのパラメータを含むので、より良い結果が出ているとも取れる。ゆえに、状態サイズ$d_{CS}=50$に設定した上で同様の結果を得られたことを示す  
計算速度に関して言えば、言語モデルの時と比較して、C2Wモデルを適用した際により低速になっていることがわかる  
これは、ネットワークのボトルネックであった語彙全体に渡るソフトマックスが存在しないことが理由として挙げられる  
しかし、このようにBi-LSTMモデルは三倍遅いが、システムの性能を著しく損なうことはないことがわかる(????????????)  

#### 複数の言語における結果
5つの言語における結果を表4に示した  
この表より、C2Wモデルが単語ルックアップテーブルを用いたモデルよりも常に良い結果であることがわかる  
また、デフォルトの特徴セットを用いたStanfordの品詞タグ付け器の結果と比較したところ、ほとんどの場合において若干優れていることを確認した  
これは、いかなる手作業もせず、同じ訓練データを用いていることを考慮すると、有望な結果であるといえる  
さらに、トルコ語では4%超の劇的な改善が見られた  

#### ベンチマークによる比較
ほとんどの最先端手法はよい辞書的特徴を学習・手作りしたり、追加のrawデータを用いて教師なしの特徴学習を行うなどして作成される  
一般に、この両方を行うことにより最適な結果が得られるとされている  
表5にPTBデータセットにおける現在のタグ付けのベンチマークを記す  
"feat"および"data"の列は、手作業による特徴が使用されているか、また追加のデータが用いられているかどうかを定義する  
この表を見ると、特徴選択や教師なしの予備学習をしなくても、C2Wモデルは現在の最先端手法と同等のレベルであることがわかる  
しかし、手作業による特徴をC2Wモデルに加えた場合、さらなる改善が見られた  
品詞を同定できるような形態素を持たへん単語はいっぱいある  
このような単語について、訓練データにラベルがないと、曖昧な文脈においてエラーに繋がる可能性がある  
skip-n-gramのような教師なし手法は、アノテートがないコーポラにおいて単語表現を事前学習することができる  
英wikipediaデータセットと構造化Skip-n-gramを用いて埋め込みを学習した  
いえーーーーーーーーーーーーいイミフ〜〜〜〜〜〜〜〜〜〜〜〜
